# Static tables: "Monday", "Tuesday"...."Sunday"
# Stop ID schedule and trip proportion for each route ID

In order to obtain a travel time prediction for a user's selected journey segment, we implement a machine learning-based algorithm which first predicts the total trip time for that route given the environmental conditions, and secondly obtain the proportion of the user's chosen segment of that total route. This notebook outlines the steps taken to compile a static table containing, for each bus route ID, the list of stops and their respective proportion from the start of the bus trip up to that stop in the one direction (route ID mirrors direction). Therefore, this table must be queried to reflect the user's chosen start and end point. The user's embarking stop proportion is subtracted from that of the disembarking stop to obtain the total segment proportion.<br><br>
&emsp;For each stop served by a given route, we also document the list of scheduled departure times for the route on that day. This allows us to build a timetable feature later on as an add-on to our application, without the need for a static table for that task alone.

In [2]:
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
from numpy import int64
from numpy import float64
from numpy import datetime64
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.dates as dates
# Allows plots to appear directly in the notebook.
%matplotlib inline
# For dealing with some Accented characters (in Irish Place names)
import unidecode
# Date/time functionality
import datetime
import time
# Check if files exist
from os.path import exists
from os import makedirs
# System specific parameters and functions
import sys
# look at some z-scores for inspecting outliers.
from scipy import stats
import seaborn as sb
# lookup lat/long and convert lat/long to national grid references.
#import geopy
#import pyproj

from patsy import dmatrices
from sklearn import metrics

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
#from sklearn.tree import export_text

from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

#import graphviz
#from graphviz import Source
#to read all CSV files in a folder
import os
import glob


# prevent columns being hidden
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:
#method to read chunks from a file and create a dataframe list.
def get_chunks(x):
    """x=[location, dtypes, datecolumns ,date_parser_func, cols]
    exp:mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
    exp:dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
    exp:cols=['Date time','Temperature','Wind Speed','Precipitation','Cloud Cover']
    exp:x=['/Chunks/DBUS/QP_Implementation',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser, cols]"""
    cwd=os.getcwd()
    path = x[0]
    chunk_folder = glob.glob(os.path.join(path, "*.csv"))
    chunk_list=[]
    counter=0
    for filename in chunk_folder:
        # read the csv file
        if len(x)==4:
            df_chunk = pd.read_csv(filename, low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3])
        else:
            df_chunk = pd.read_csv(filename, usecols=x[4], low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3])
        chunk_list.append(df_chunk)
        print('Index at chunk_list',counter,' is ', filename, ': ',df_chunk.shape)
        counter+=1
    return chunk_list

#chunk_dbus_list=get_chunks(arg_list)

In [4]:
# get the timetable files
def getTimetableChunks(x):
    """x=[location, dtypes, datecolumns ,date_parser_func, cols]
    exp:mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
    exp:dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
    exp:cols=['Date time','Temperature','Wind Speed','Precipitation','Cloud Cover']
    exp:x=['/Chunks/DBUS/QP_Implementation',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser, cols]"""
    cwd=os.getcwd()
    path = x[0]
    chunk_folder = glob.glob(os.path.join(path, "*.csv"))
    chunk_list=[]
    counter=0
    for filename in chunk_folder:
        # read the csv file
        df_chunk = pd.read_csv(filename, low_memory=False, dtype=x[1], index_col=0)
        chunk_list.append(df_chunk)
        print('Index at chunk_list',counter,' is ', filename, ': ',df_chunk.shape)
        counter+=1
    return chunk_list

In [5]:
def getDayTimetable(df, day):
    """Creates a timetable for a given weekday on a given route dataframe."""
    df = df[df.DAY_OF_WEEK == day]
    df_max_PROGRNUMBER=pd.DataFrame(df.groupby(['TRIPID'])['PROGRNUMBER'].max())
    df_max_PROGRNUMBER.rename(columns={'PROGRNUMBER':'PROGRNUMBER_MAX'},inplace=True)
    df = pd.merge(df, df_max_PROGRNUMBER, on="TRIPID", how="left")
    max_PROGRNUMBER = np.amax(df['PROGRNUMBER'].unique())
    df.drop(df[df.PROGRNUMBER_MAX != max_PROGRNUMBER].index, inplace=True)
    target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
    target.dropna(how='any', inplace=True)
    this_route = target['ROUTEID'].unique()[0]
    # convert planned dep time to int
    target = target.astype({'PLANNEDTIME_DEP':'int32'})
    
    # write to new csv in timetable folder
    target.to_csv(f'../timetables_V3/{this_route}_{day}.csv')

## 1 Read Chunks

Each route's dataset is found in the IMP_ROUTE_ID folder, which contains routes that operated throughout the year (temporary routes filtered out as outliers). Hourly weather data also imported in this file.

In [6]:
dtype={'TRIPID' :'category',
       'PROGRNUMBER':'int16',
       'STOPPOINTID':'category',
       'PLANNEDTIME_ARR':'int64',
       'PLANNEDTIME_DEP':'int64',
       'ACTUALTIME_ARR':'int64',
       'ACTUALTIME_DEP':'int64',
       'DIFFERENCETIME_ARR':'int64',
       'DIFFERENCETIME_DEP':'int64',
       'ROUTEID':'category',
       'TOTAL_JOURNEY_TIME':'int16',
       'TIME_OF_DAY':'category',
       'DAY_OF_WEEK':'category',
       'TRIPS_TIME_PROPORTION_v2':'float32',
       'PLANNEDTIME_DEP_R_M5':'category',
       'rain':'float16',
       'temp':'float16',
       'wet_bulb_temp(C)':'float16',
       'dew_pt_temp(C)':'float16',
       'vapour_pressure(hPa)':'float16',
       'humidity(%)':'int16',
       'sea_lvl_pressure(hPa)':'float16'}

mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")


data_list = ["../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/", dtype, ['ACTUALTIME_ARR_DATETIME'], mydateparser]


In [7]:
chunks = get_chunks(data_list)

Index at chunk_list 0  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/18_3.csv :  (826557, 25)
Index at chunk_list 1  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/67_6.csv :  (775062, 25)
Index at chunk_list 2  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/142_9.csv :  (23881, 25)
Index at chunk_list 3  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/17_16.csv :  (29666, 25)
Index at chunk_list 4  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/46A_74.csv :  (2094278, 25)
Index at chunk_list 5  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/122_14.csv :  (427306, 25)
Index at chunk_list 6  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/150_8.csv :  (524122, 25)
Index at chunk_list 7  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/33X_49.csv :  (56923, 25)
Index at chunk_list 8  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/83_19.csv :  (96394, 25)
Index at chunk_list 9  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/270_44.csv :  (100767, 25)
Index at chunk_list 10  is  ../../Pelin/C

Index at chunk_list 85  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/7_73.csv :  (135923, 25)
Index at chunk_list 86  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/69_47.csv :  (214862, 25)
Index at chunk_list 87  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/13_66.csv :  (114180, 25)
Index at chunk_list 88  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/32_58.csv :  (448425, 25)
Index at chunk_list 89  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/145_91.csv :  (77747, 25)
Index at chunk_list 90  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/66_13.csv :  (223485, 25)
Index at chunk_list 91  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/185_56.csv :  (63446, 25)
Index at chunk_list 92  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/14_12.csv :  (24388, 25)
Index at chunk_list 93  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/83A_23.csv :  (256801, 25)
Index at chunk_list 94  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/53_20.csv :  (73975, 25)
Index at chunk_list 95  is  ../.

Index at chunk_list 169  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/41B_53.csv :  (58874, 25)
Index at chunk_list 170  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/53_23.csv :  (5724, 25)
Index at chunk_list 171  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/44_36.csv :  (448610, 25)
Index at chunk_list 172  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/61_106.csv :  (246711, 25)
Index at chunk_list 173  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/41C_78.csv :  (740820, 25)
Index at chunk_list 174  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/65B_66.csv :  (347417, 25)
Index at chunk_list 175  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/49_27.csv :  (475452, 25)
Index at chunk_list 176  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/38_14.csv :  (527184, 25)
Index at chunk_list 177  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/18_4.csv :  (783294, 25)
Index at chunk_list 178  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/39A_42.csv :  (32949, 25)
Index at chunk_list 

Index at chunk_list 253  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/33_72.csv :  (304625, 25)
Index at chunk_list 254  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/38A_33.csv :  (62279, 25)
Index at chunk_list 255  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/15_16.csv :  (1532473, 25)
Index at chunk_list 256  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/7B_93.csv :  (70065, 25)
Index at chunk_list 257  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/27A_4.csv :  (378802, 25)
Index at chunk_list 258  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/59_10.csv :  (99918, 25)
Index at chunk_list 259  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/75_17.csv :  (622089, 25)
Index at chunk_list 260  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/44B_60.csv :  (32940, 25)
Index at chunk_list 261  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/13_67.csv :  (767187, 25)
Index at chunk_list 262  is  ../../Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/66_17.csv :  (403689, 25)
Index at chunk_list 2

In [8]:
# remove some of the weather data not available in api
for df in chunks:
    df.drop(['wet_bulb_temp(C)', 'vapour_pressure(hPa)'], axis=1, inplace=True)
    
list(chunks[0])

['TRIPID',
 'PROGRNUMBER',
 'STOPPOINTID',
 'PLANNEDTIME_ARR',
 'PLANNEDTIME_DEP',
 'ACTUALTIME_ARR',
 'ACTUALTIME_DEP',
 'DIFFERENCETIME_ARR',
 'DIFFERENCETIME_DEP',
 'ROUTEID',
 'ACTUALTIME_ARR_DATETIME',
 'TOTAL_JOURNEY_TIME',
 'TIME_OF_DAY',
 'DAY_OF_WEEK',
 'TRIPS_TIME_PROPORTION_v2',
 'DAYOFSERVICE',
 'PLANNEDTIME_DEP_DATETIME',
 'PLANNEDTIME_DEP_R_M5',
 'rain',
 'temp',
 'dew_pt_temp(C)',
 'humidity(%)',
 'sea_lvl_pressure(hPa)']

## 2 Obtaining timetable and proportions for each route's stop sequence

In [18]:
# want to drop any route dataframes if their r2 was below the acceptance threshold
drop_routes = pd.read_csv('../../Pelin/ML_route_bel_tres_R2_cor01_df.csv')
drop = drop_routes['ROUTEID'].unique()

count = 0
for df in chunks:
    if df['ROUTEID'].unique()[0] in drop:
        chunks.pop(count)
    count += 1
        

In [19]:
# confirm that the 6 routes have been dropped 
len(chunks)

299

#### Daily timetables obtained for improved query time

In [20]:
for df in chunks:
    
    try:
        getDayTimetable(df, "Monday")
    except ValueError:
        # value error occurs if that route does not run on the day specified
        pass
    try:
        getDayTimetable(df, "Tuesday")
    except ValueError:
        pass
    try:
        getDayTimetable(df, "Wednesday")
    except ValueError:
        pass
    try:
        getDayTimetable(df, "Thursday")
    except ValueError:
        pass
    try:
        getDayTimetable(df, "Friday")
    except ValueError:
        pass
    
    try:
        getDayTimetable(df, "Saturday")
    except ValueError:
        pass
        
    try:
        getDayTimetable(df, "Sunday")
    except ValueError:
        pass

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

...

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  target=(df.groupby(['ROUTEID','DAY_OF_WEEK','STOPPOINTID' , 'PLANNEDTIME_DEP_R_M5'], as_index=False))['TRIPS_TIME_PROPORTION_v2', 'PLANNEDTIME_DEP'].mean()
/tmp/ipykernel_10141/2668783288.py:9:

In [25]:
dtypes={'STOPPOINTID':'category',
       'PLANNEDTIME_DEP':'int64',
       'ROUTEID':'category',
       'DAY_OF_WEEK':'category',
       'TRIPS_TIME_PROPORTION_v2':'float32',
       'PLANNEDTIME_DEP_R_M5':'category'}

location = '../timetables_V3'
data = [location, dtypes]

# commenting this out as will use yield function instead
#timetable_chunks = getTimetableChunks(data)

In [26]:
#method to yield(return generator object) dfs for memory efficent concat( to avoid expotential memory use)
#https://stackoverflow.com/questions/70911236/how-can-i-concat-the-huge-pandas-dataframes-into-one
def yield_dfs(x):
    path = x[0]
    file_list = glob.glob(os.path.join(path, "*.csv"))
    """generator function to yield dataframes."""
    for filename in file_list:
        df = pd.read_csv(filename, low_memory=False, dtype=x[1], index_col=False)
        print(df.shape)
        yield df

In [27]:
timetables_df = pd.concat(yield_dfs(data), axis=0,ignore_index=True)

(558, 7)
(231, 7)
(386, 7)
(1562, 7)
(242, 7)
(2056, 7)
(2098, 7)
(9576, 7)
(1948, 7)
(696, 7)
(2008, 7)
(11000, 7)
(3251, 7)
(1371, 7)
(1758, 7)
(11705, 7)
(751, 7)
(526, 7)
(7486, 7)
(6490, 7)
(253, 7)
(519, 7)
(6965, 7)
(640, 7)
(2116, 7)
(2683, 7)
(694, 7)
(248, 7)
(1358, 7)
(733, 7)
(15882, 7)
(571, 7)
(1146, 7)
(9158, 7)
(2762, 7)
(3847, 7)
(4510, 7)
(4291, 7)
(254, 7)
(5736, 7)
(2033, 7)
(662, 7)
(1314, 7)
(518, 7)
(1774, 7)
(591, 7)
(1173, 7)
(3068, 7)
(6422, 7)
(7708, 7)
(1953, 7)
(3887, 7)
(902, 7)
(7425, 7)
(5963, 7)
(2227, 7)
(2476, 7)
(5994, 7)
(402, 7)
(4810, 7)
(420, 7)
(299, 7)
(2954, 7)
(4387, 7)
(1353, 7)
(2223, 7)
(2787, 7)
(527, 7)
(1376, 7)
(211, 7)
(231, 7)
(654, 7)
(539, 7)
(3861, 7)
(13279, 7)
(4072, 7)
(5149, 7)
(13524, 7)
(46, 7)
(1862, 7)
(475, 7)
(4852, 7)
(7054, 7)
(192, 7)
(39, 7)
(2043, 7)
(839, 7)
(5370, 7)
(3056, 7)
(4039, 7)
(1865, 7)
(1045, 7)
(151, 7)
(314, 7)
(1517, 7)
(1041, 7)
(1804, 7)
(1750, 7)
(570, 7)
(2001, 7)
(5592, 7)
(5570, 7)
(1527, 7)
(4

(9422, 7)
(902, 7)
(3352, 7)
(5069, 7)
(1832, 7)
(2195, 7)
(2375, 7)
(764, 7)
(3529, 7)
(2035, 7)
(3554, 7)
(1110, 7)
(261, 7)
(17086, 7)
(3556, 7)
(6604, 7)
(1261, 7)
(6412, 7)
(2366, 7)
(484, 7)
(20, 7)
(2321, 7)
(172, 7)
(11647, 7)
(1579, 7)
(808, 7)
(7121, 7)
(789, 7)
(7259, 7)
(2276, 7)
(12334, 7)
(3212, 7)
(12391, 7)
(4237, 7)
(3054, 7)
(515, 7)
(4358, 7)
(739, 7)
(6482, 7)
(1749, 7)
(2611, 7)
(372, 7)
(1887, 7)
(52, 7)
(1193, 7)
(5556, 7)
(2612, 7)
(2926, 7)
(5150, 7)
(193, 7)
(5693, 7)
(558, 7)
(742, 7)
(1987, 7)
(2522, 7)
(2536, 7)
(393, 7)
(1879, 7)
(258, 7)
(2343, 7)
(5345, 7)
(1767, 7)
(2563, 7)
(1747, 7)
(206, 7)
(571, 7)
(1913, 7)
(5413, 7)
(3510, 7)
(6747, 7)
(11668, 7)
(4786, 7)
(2362, 7)
(75, 7)
(741, 7)
(1575, 7)
(324, 7)
(7344, 7)
(2124, 7)
(5250, 7)
(1271, 7)
(4379, 7)
(3134, 7)
(1012, 7)
(555, 7)
(4965, 7)
(299, 7)
(6644, 7)
(601, 7)
(4596, 7)
(1139, 7)
(3478, 7)
(738, 7)
(804, 7)
(483, 7)
(4900, 7)
(428, 7)
(1846, 7)
(5678, 7)
(240, 7)
(3015, 7)
(346, 7)
(904, 7)


(904, 7)
(784, 7)
(12975, 7)
(1910, 7)
(3882, 7)
(3472, 7)
(2606, 7)
(1467, 7)
(3043, 7)
(10093, 7)
(5031, 7)
(982, 7)
(4162, 7)
(5779, 7)
(346, 7)
(2190, 7)
(1299, 7)
(1444, 7)
(2430, 7)
(2297, 7)
(198, 7)
(632, 7)
(2461, 7)
(2715, 7)
(1722, 7)
(2093, 7)
(6357, 7)
(5701, 7)
(285, 7)
(528, 7)
(1768, 7)
(2616, 7)
(6009, 7)
(4614, 7)
(3663, 7)
(6721, 7)
(6715, 7)
(2339, 7)
(11220, 7)
(4920, 7)
(2140, 7)
(573, 7)
(1535, 7)
(1938, 7)
(622, 7)
(4640, 7)
(21, 7)
(4897, 7)
(3047, 7)
(2303, 7)
(4491, 7)
(659, 7)
(6206, 7)
(3402, 7)
(1961, 7)
(983, 7)
(738, 7)
(552, 7)
(88, 7)
(1874, 7)
(4228, 7)
(3960, 7)
(1368, 7)
(4417, 7)
(4996, 7)
(3096, 7)
(1997, 7)
(4872, 7)
(4052, 7)
(1721, 7)
(7674, 7)
(10187, 7)
(4621, 7)
(145, 7)
(6583, 7)
(7503, 7)
(3883, 7)
(1069, 7)
(3654, 7)
(1133, 7)
(2317, 7)
(314, 7)
(127, 7)
(1769, 7)
(5334, 7)
(4498, 7)
(210, 7)
(467, 7)
(7292, 7)
(5709, 7)
(913, 7)
(958, 7)
(2484, 7)
(2723, 7)
(12026, 7)
(784, 7)
(2328, 7)
(12332, 7)
(8374, 7)
(5462, 7)
(5680, 7)
(1099, 7)


In [30]:
timetables_df.shape

(6563578, 7)

In [28]:
timetables_df.to_csv('timetables_V3.csv', index=False)